In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [3]:
from src.dataloader import *
from src.models import *
from src.trainer import *
from src.utils import *

In [4]:
DATADRIVE = '/datadrive_ssd/'

In [5]:
!ls /datadrive_ssd/tigge/32km/total_precipitation_ens10/

2019-10.nc  2020-01.nc	2020-04.nc  2020-07.nc	2020-10.nc
2019-11.nc  2020-02.nc	2020-05.nc  2020-08.nc	2020-11.nc
2019-12.nc  2020-03.nc	2020-06.nc  2020-09.nc


In [35]:
!ls /datadrive_ssd/tigge/32km/*ens10

/datadrive_ssd/tigge/32km/2m_temperature_ens10:
2020-06.nc

/datadrive_ssd/tigge/32km/total_column_water_ens10:

/datadrive_ssd/tigge/32km/total_precipitation_ens10:
2019-10.nc  2020-01.nc	2020-04.nc  2020-07.nc	2020-10.nc
2019-11.nc  2020-02.nc	2020-05.nc  2020-08.nc	2020-11.nc
2019-12.nc  2020-03.nc	2020-06.nc  2020-09.nc


In [36]:
# Dummy data to test
ds_train = TiggeMRMSDataset(
    tigge_dir=f'{DATADRIVE}/tigge/32km/',
    tigge_vars=['total_precipitation_ens10', '2m_temperature'],
    mrms_dir=f'{DATADRIVE}/mrms/4km/RadarOnly_QPE_06H/',
    rq_fn=f'{DATADRIVE}/mrms/4km/RadarQuality.nc',
#     const_fn='/datadrive/tigge/32km/constants.nc',
#     const_vars=['orog', 'lsm'],
    data_period=('2020-01', '2020-01'),
    first_days=5,
)

/anaconda/envs/nwp-downscale/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/anaconda/envs/nwp-downscale/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Loading data


In [39]:
ds_train.tigge.isel(variable=1, valid_time=0, lat=0, lon=0)

<xarray.DataArray (member: 10)>
array([0.57756295, 0.57756295, 0.57756295, 0.57756295, 0.57756295,
       0.57756295, 0.57756295, 0.57756295, 0.57756295, 0.57756295])
Coordinates:
    lead_time   timedelta64[ns] 12:00:00
    init_time   datetime64[ns] 2020-01-01
  * member      (member) int32 0 1 2 3 4 5 6 7 8 9
    lon         float64 235.2
    lat         float64 49.68
    valid_time  datetime64[ns] 2020-01-01T12:00:00
    variable    <U3 't2m'

In [29]:
ds_train.ensemble_mode = 'random'

In [30]:
X, y = ds_train[0]

In [31]:
X.shape

(1, 16, 16)

In [14]:
np.random.choice(ds_train.tigge.member)

2

In [21]:
ds_train.tigge

<xarray.DataArray (variable: 1, member: 10, valid_time: 9, lat: 93, lon: 172)>
array([[[[[2.18321646e-03, 8.49898932e-03, 3.70182903e-02, ...,
           6.68738970e-03, 6.36622879e-03, 6.04994385e-03],
          [3.67798838e-03, 2.13319347e-03, 3.19674820e-04, ...,
           5.23660214e-03, 5.07866091e-03, 6.23173361e-03],
          [6.48309798e-02, 4.73257884e-02, 1.55569814e-02, ...,
           4.30885125e-03, 4.58184345e-03, 5.32891772e-03],
          ...,
          [0.00000000e+00, 0.00000000e+00, 2.85384505e-05, ...,
           5.92190308e-05, 8.43766415e-07, 3.08053266e-06],
          [8.15085159e-05, 1.86770472e-04, 6.44181333e-04, ...,
           0.00000000e+00, 1.44217295e-05, 6.57764689e-05],
          [2.25996044e-04, 4.88405712e-04, 6.09514745e-04, ...,
           1.80209716e-03, 1.37572896e-03, 2.42994596e-03]],

         [[1.24033787e-02, 2.39887774e-03, 4.02094039e-03, ...,
           1.56565983e-03, 1.27659235e-03, 1.04587243e-03],
          [8.73725506e-04, 2.26283276e-04, 3.74594827e-05, ...,
           2.09303955e-03, 1.60421720e-03, 1.00376678e-03],
          [1.17575451e-02, 4.60401322e-03, 1.51394603e-03, ...,
           2.62565518e-03, 2.02795571e-03, 1.18024985e-03],
...
          [5.40422327e-03, 7.37761615e-03, 7.26241410e-03, ...,
           1.88710132e-04, 2.27902952e-05, 0.00000000e+00],
          [3.92260682e-03, 7.11693417e-03, 1.73406479e-03, ...,
           1.59428609e-03, 2.01030197e-04, 8.24236632e-06],
          [2.61869084e-03, 1.21929000e-03, 1.46141856e-03, ...,
           1.31815412e-03, 6.06405123e-03, 4.67621233e-03]],

         [[4.39775797e-02, 4.38860554e-02, 3.73230496e-02, ...,
           3.49868303e-04, 3.59768831e-04, 2.75648560e-04],
          [5.87470013e-02, 4.55067799e-02, 3.90551222e-02, ...,
           8.32443211e-04, 1.00198146e-03, 8.87889058e-04],
          [8.60909322e-02, 7.18875474e-02, 5.24891357e-02, ...,
           1.44545500e-03, 1.85510391e-03, 1.70368614e-03],
          ...,
          [6.46445540e-05, 6.90369595e-05, 0.00000000e+00, ...,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [1.22973293e-03, 1.06080156e-04, 1.12199085e-04, ...,
           1.71272243e-05, 9.83025793e-05, 1.33122809e-05],
          [1.77407048e-03, 3.78705516e-04, 5.72203778e-05, ...,
           8.82182240e-05, 2.37138776e-04, 3.63108048e-04]]]]])
Coordinates:
    lead_time   timedelta64[ns] 12:00:00
    init_time   (valid_time) datetime64[ns] 2020-01-01 ... 2020-01-05
  * member      (member) int32 0 1 2 3 4 5 6 7 8 9
  * lon         (lon) float64 235.2 235.5 235.8 236.2 ... 289.3 289.6 289.9
  * lat         (lat) float64 49.68 49.36 49.04 48.72 ... 21.2 20.88 20.56 20.24
  * valid_time  (valid_time) datetime64[ns] 2020-01-01T12:00:00 ... 2020-01-0...
  * variable    (variable) <U2 'tp'

In [25]:
ds_train.tigge.rename({'variable': 'raw_variable'}).stack(variable = ['raw_variable', 'member']).transpose(
    'variable', 'valid_time', 'lat', 'lon')

<xarray.DataArray (variable: 10, valid_time: 9, lat: 93, lon: 172)>
array([[[[2.18321646e-03, 8.49898932e-03, 3.70182903e-02, ...,
          6.68738970e-03, 6.36622879e-03, 6.04994385e-03],
         [3.67798838e-03, 2.13319347e-03, 3.19674820e-04, ...,
          5.23660214e-03, 5.07866091e-03, 6.23173361e-03],
         [6.48309798e-02, 4.73257884e-02, 1.55569814e-02, ...,
          4.30885125e-03, 4.58184345e-03, 5.32891772e-03],
         ...,
         [0.00000000e+00, 0.00000000e+00, 2.85384505e-05, ...,
          5.92190308e-05, 8.43766415e-07, 3.08053266e-06],
         [8.15085159e-05, 1.86770472e-04, 6.44181333e-04, ...,
          0.00000000e+00, 1.44217295e-05, 6.57764689e-05],
         [2.25996044e-04, 4.88405712e-04, 6.09514745e-04, ...,
          1.80209716e-03, 1.37572896e-03, 2.42994596e-03]],

        [[1.24033787e-02, 2.39887774e-03, 4.02094039e-03, ...,
          1.56565983e-03, 1.27659235e-03, 1.04587243e-03],
         [8.73725506e-04, 2.26283276e-04, 3.74594827e-05, ...,
          2.09303955e-03, 1.60421720e-03, 1.00376678e-03],
         [1.17575451e-02, 4.60401322e-03, 1.51394603e-03, ...,
          2.62565518e-03, 2.02795571e-03, 1.18024985e-03],
...
         [5.40422327e-03, 7.37761615e-03, 7.26241410e-03, ...,
          1.88710132e-04, 2.27902952e-05, 0.00000000e+00],
         [3.92260682e-03, 7.11693417e-03, 1.73406479e-03, ...,
          1.59428609e-03, 2.01030197e-04, 8.24236632e-06],
         [2.61869084e-03, 1.21929000e-03, 1.46141856e-03, ...,
          1.31815412e-03, 6.06405123e-03, 4.67621233e-03]],

        [[4.39775797e-02, 4.38860554e-02, 3.73230496e-02, ...,
          3.49868303e-04, 3.59768831e-04, 2.75648560e-04],
         [5.87470013e-02, 4.55067799e-02, 3.90551222e-02, ...,
          8.32443211e-04, 1.00198146e-03, 8.87889058e-04],
         [8.60909322e-02, 7.18875474e-02, 5.24891357e-02, ...,
          1.44545500e-03, 1.85510391e-03, 1.70368614e-03],
         ...,
         [6.46445540e-05, 6.90369595e-05, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.22973293e-03, 1.06080156e-04, 1.12199085e-04, ...,
          1.71272243e-05, 9.83025793e-05, 1.33122809e-05],
         [1.77407048e-03, 3.78705516e-04, 5.72203778e-05, ...,
          8.82182240e-05, 2.37138776e-04, 3.63108048e-04]]]])
Coordinates:
    lead_time     timedelta64[ns] 12:00:00
    init_time     (valid_time) datetime64[ns] 2020-01-01 ... 2020-01-05
  * lon           (lon) float64 235.2 235.5 235.8 236.2 ... 289.3 289.6 289.9
  * lat           (lat) float64 49.68 49.36 49.04 48.72 ... 20.88 20.56 20.24
  * valid_time    (valid_time) datetime64[ns] 2020-01-01T12:00:00 ... 2020-01...
  * variable      (variable) MultiIndex
  - raw_variable  (variable) object 'tp' 'tp' 'tp' 'tp' ... 'tp' 'tp' 'tp' 'tp'
  - member        (variable) int64 0 1 2 3 4 5 6 7 8 9